In [110]:
#currencey parsing
import requests
from xml.etree import ElementTree as ET
from datetime import datetime, date, time,timedelta
import pandas as pd
import numpy as np

local_data = False
file_name = 'filename_test.xml'
start_date = datetime.strptime("2001-1-1", "%Y-%W-%w")
end_date = datetime.strptime("2001-3-1", "%Y-%W-%w") #not less than last date of data
url='http://www.cbr.ru/scripts/XML_dynamic.asp?date_req1=%s&date_req2=%s&VAL_NM_RQ=R01235'%(start_date.strftime("%d/%m/%Y"),end_date.strftime("%d/%m/%Y"))




In [117]:
def lookup(s): #VERY FAST
    dates = {date:pd.to_datetime(date,format='%d.%m.%Y') for date in s.unique()}
    return s.map(dates)

if local_data:
    #get data from file
    tree = ET.parse(file_name)
    root = tree.getroot()
else:
    #from server and save to file
    response = requests.get(url)
    root = ET.fromstring(response.content)
    tree = ET.ElementTree(root)
    tree.write(file_name)

curr_arr = pd.DataFrame({'date': list([x.attrib['Date'] for x in root]),'cv': list(map(lambda s: s.replace(',' , '.'), [x[1].text for x in root]))})
curr_arr['date'] = lookup(curr_arr['date'])
curr_arr['cv'] = curr_arr['cv'].astype(float)
curr_arr = curr_arr.set_index('date')
curr_arr = curr_arr.reindex(pd.date_range(start_date,end_date))
curr_arr = curr_arr.fillna(method='ffill')
curr_arr = curr_arr.fillna(method='bfill')
curr_arr['cd'] = curr_arr['cv'].diff(1)
curr_arr['cd'] = curr_arr['cd'].fillna(method='bfill')
curr_arr['cd'] = curr_arr['cd'].rolling(7).mean()
curr_arr['cd'] = curr_arr['cd'].fillna(0)
curr_arr

,cv,cd
2001-01-01,28.16,0.000000
2001-01-02,28.16,0.000000
2001-01-03,28.16,0.000000
2001-01-04,28.16,0.000000
2001-01-05,28.48,0.000000
2001-01-06,28.43,0.000000
2001-01-07,28.43,0.038571
2001-01-08,28.43,0.038571
2001-01-09,28.43,0.038571
2001-01-10,28.39,0.032857
